In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np. random.seed(100)

In [1]:
#  Neuron class = calculation of: output, activation potential, activation functions
class activation_fcn(object):

    def __init__(self):
        pass

    # Claculate neuron output
    def output(self, layer, name, derivative=False):
        # Get the method from 'self'. Default to a lambda.
        method = getattr(self, name, "Invalid_function")
        # Call the method as we return it
        return method(layer, derivative)

    # Display error if wrong activation function name is selected
    def Invalid_function(self, *arg):
        print("Error: Invalid activation function")
        return None

    # Identity activation function
    def linear(self, layer, derivative=False):
        print(layer)
        if not derivative:
            return layer
        else:
            return np.ones(shape=np.shape(layer))

    def tanh(self, layer, derivative=False):
        if not derivative:
            return np.tanh(layer)
        else:
            return 1 - np.tanh(layer)**2

    def logistic(self, layer, derivative=False):
        out = 0
        if not derivative:
            out = 1/(1 + np.exp(-layer))
        else:
            out = layer * (1-layer)
        return out

    def relu(self, layer, derivative=False):
        if not derivative:
            return np.maximum(0, layer)
        else:
            return np.where(layer > 0, 1, 0)


In [ ]:
#  Loss function class
class loss_fcn(object):

    def __init__(self):
        pass

    # Loss/error value calculated for all input data sample
    def loss(self, loss, expected, outputs, derivative):
        # Get the method from 'self'. Default to a lambda.
        method = getattr(self, str(loss), lambda expected, outputs, derivative: "Invalid loss function")
        # Call the method as we return it

        return method(expected, outputs, derivative)

    # Display error if wrong loss function name is selected
    def Invalid_function(self, *arg):
        print("Error: Invalid loss function")
        return None

    # Mean Square Error loss function
    def mse(self, expected, outputs, derivative=False):
        error_value = 0
        if not derivative:
            error_value = 0.5 * np.power(expected - outputs, 2)
        else:
            error_value = -(expected - outputs)
        return error_value

    # Cross Entorpy loss function
    def binary_cross_entropy(self, expected, outputs, derivative=False):
        error_value = 0
        if not derivative:
            error_value = -(expected * np.log(outputs) + (1 - expected) * np.log(1 - outputs))
        else:
            error_value = -(expected / outputs) + (1 - expected) / (1 - outputs)
        return error_value